In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
DEVICE = torch.device("cuda:0")
# Moving to the gpu: net = Net().to(DEVICE)

In [5]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        encode = []
        relu = nn.ReLU(True)
        pool = nn.MaxPool2d(2)
        encode.append(nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(32, 64, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(128, 256, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(256, 256, 3, padding=1), relu))
        
        self.encoders = []
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for n-1
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for D(n-1)
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for n
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for D(n+1)
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for n+1
        
        decode = []
        decode.append(nn.Sequential(nn.Conv2d(256*5, 256, 3, padding = 1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(256*6, 128, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(128*6, 64, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(64*6, 32, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(32*6, 1, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        
        self.decoder = nn.Sequential(*decode).to(DEVICE)
        '''
        transform = []
        transform.append(nn.Sequential(nn.Conv3d(1, 32, 3), relu))
        transform.append(nn.Sequential(nn.Conv2d(32, 1, 3), relu))
        
        self.transformer = nn.Sequential(*transform)
        '''
    def forward(self, x):
        '''Fuction uses skips parameter to
        keep track of skip connections between
        encoder and decoder layers.'''
        skips = [[],[],[],[],[]]
        nextinput = x
        for i, encoder in enumerate(self.encoders):
            x = nextinput
            for layer in encoder:
                x = layer(x)
                skips[i].append(x)
        x = skips[-1][-1]
        for i, layer in enumerate(self.decoder):
            for j, encoder in enumerate(skips):
                if j == len(skips)-1 and i == 0: break
                x = torch.cat((x,encoder[-(i+1)]),1)
            x = layer(x)
        return x

network = autoencoder()
network = network.to(DEVICE)
x = torch.randn(1,1280,720).to(DEVICE)
network(x.view(-1,1,1280,720)).size()


torch.Size([1, 1, 1280, 720])